## Ejemplo Modelo Seq2Seq Sin atención

In [56]:
def quitarchar(texto):
  texto = texto.replace("¡", "").replace("¿", "")
  return texto.strip()

with open('spa.txt','r',encoding='utf8') as f:
  datos=[(quitarchar(x.split('\t')[0]),quitarchar(x.split('\t')[1])) for x in f.readlines()]

In [57]:
datos[:10]

[('Go.', 'Ve.'),
 ('Go.', 'Vete.'),
 ('Go.', 'Vaya.'),
 ('Go.', 'Váyase.'),
 ('Hi.', 'Hola.'),
 ('Run!', 'Corre!'),
 ('Run!', 'Corran!'),
 ('Run!', 'Huye!'),
 ('Run!', 'Corra!'),
 ('Run!', 'Corred!')]

In [58]:
import numpy as np
np.random.shuffle(datos)
palabras_ingles, palabras_español = zip(*datos)
long_vocab_ingles = len(palabras_ingles)
long_vocab_español = len(palabras_español)

In [59]:
print(f'Longitud Vocab Español: {long_vocab_ingles}')
print(f'Logitud Vocab Inglés: {long_vocab_español}')

Longitud Vocab Español: 141543
Logitud Vocab Inglés: 141543


In [60]:
for i in range(5):
    print(palabras_ingles[i], "=>", palabras_español[i])

I'm single again. => Estoy solo otra vez.
Don't leave your belongings unattended at the beach. => No dejes tus pertenencias desatendidas en el playa.
You're joking, aren't you? => Estás bromeando, verdad?
Why take a chance? => Para qué correr el riesgo?
There are many more students in the classroom today than yesterday. => Hoy hay muchos más estudiantes en la sala que ayer.


In [61]:
long_ing_max = max(len(linea.split()) for linea in palabras_ingles)
long_esp_max = max(len(linea.split()) for linea in palabras_español)
long_secuencia = max(long_ing_max, long_esp_max)

print(f'Max Longitud Frases en Inglés: {long_ing_max}')
print(f'Max Longitud Frases en Español: {long_esp_max}')
print(f'Logitud de la secuencia: {long_secuencia}')

Max Longitud Frases en Inglés: 70
Max Longitud Frases en Español: 68
Logitud de la secuencia: 70


In [62]:
import tensorflow as tf
from keras.layers import TextVectorization

tam_voc = 1000
long_max = 70
capa_vect_ing = tf.keras.layers.TextVectorization(
    tam_voc, output_sequence_length=long_max)
capa_vect_esp = tf.keras.layers.TextVectorization(
    tam_voc, output_sequence_length=long_max)
capa_vect_ing.adapt(palabras_ingles)
capa_vect_esp.adapt([f"startseq {s} endseq" for s in palabras_español])

In [63]:
print("Inglés:")
print(capa_vect_ing.get_vocabulary()[:10])

print("\nEspañol:")
print(capa_vect_esp.get_vocabulary()[:10])

Inglés:
['', '[UNK]', 'i', 'the', 'to', 'you', 'tom', 'a', 'is', 'he']

Español:
['', '[UNK]', 'startseq', 'endseq', 'de', 'que', 'no', 'tom', 'a', 'la']


In [64]:
X_train = tf.constant(palabras_ingles[:120_000])
X_valid = tf.constant(palabras_ingles[120_000:])
X_train_dec = tf.constant([f"startseq {s}" for s in palabras_español[:120_000]])
X_valid_dec = tf.constant([f"startseq {s}" for s in palabras_español[120_000:]])
Y_train = capa_vect_esp([f"{s} endseq" for s in palabras_español[:120_000]])
Y_valid = capa_vect_esp([f"{s} endseq" for s in palabras_español[120_000:]])

In [65]:
ent_cod = tf.keras.layers.Input(shape=[], dtype=tf.string)
ent_dec = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [66]:
tam_emb = 128
ids_ent_cod = capa_vect_ing(ent_cod)
ids_ent_dec = capa_vect_esp(ent_dec)

capa_embedding_cod = tf.keras.layers.Embedding(tam_voc, tam_emb,
                                                    mask_zero=True)
capa_embedding_dec = tf.keras.layers.Embedding(tam_voc, tam_emb,
                                                    mask_zero=True)
embeddings_cod = capa_embedding_cod(ids_ent_cod)
embeddings_dec = capa_embedding_dec(ids_ent_dec)

In [67]:
#creamos el codificador
codificador = tf.keras.layers.LSTM(512, return_state=True)
salidas_cod, *estado_cod = codificador(embeddings_cod)

In [68]:
#creamos el decodificador
decodificador = tf.keras.layers.LSTM(512, return_sequences=True)
salidas_dec = decodificador(embeddings_dec, initial_state=estado_cod)

In [70]:
capa_salida = tf.keras.layers.Dense(tam_voc, activation="softmax")
Y_proba = capa_salida(salidas_dec)

In [72]:
modelo = tf.keras.Model(inputs=[ent_cod, ent_dec],
                       outputs=[Y_proba])
modelo.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
modelo.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None)                 │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_7             │ (None)                 │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ text_vectorization_6      │ (None, 70)             │              0 │ input_layer_6[0][0]    │
│ (TextVectorization)       │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ text_vectorization_7      │ (None, 70)             │              0 │ input_layer_7[0][0]    │
│ (TextVectorization)       │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_8 (Embedding)   │ (None, 70, 128)        │        128,000 │ text_vectorization_6[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_6 (NotEqual)    │ (None, 70)             │              0 │ text_vectorization_6[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_9 (Embedding)   │ (None, 70, 128)        │        128,000 │ text_vectorization_7[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ [(None, 512), (None,   │      1,312,768 │ embedding_8[0][0],     │
│                           │ 512), (None, 512)]     │                │ not_equal_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 70, 512)        │      1,312,768 │ embedding_9[0][0],     │
│                           │                        │                │ lstm_6[0][1],          │
│                           │                        │                │ lstm_6[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 70, 1000)       │        513,000 │ lstm_7[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,394,536 (12.95 MB)

 Trainable params: 3,394,536 (12.95 MB)

 Non-trainable params: 0 (0.00 B)

In [73]:
modelo.fit((X_train, X_train_dec), Y_train, epochs=5,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 103s 27ms/step - accuracy: 0.0375 - loss: 3.4009 - val_accuracy: 0.0549 - val_loss: 2.0048
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 100s 27ms/step - accuracy: 0.0576 - loss: 1.8417 - val_accuracy: 0.0633 - val_loss: 1.5206
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 142s 27ms/step - accuracy: 0.0658 - loss: 1.3808 - val_accuracy: 0.0665 - val_loss: 1.3532
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 147s 28ms/step - accuracy: 0.0707 - loss: 1.1399 - val_accuracy: 0.0678 - val_loss: 1.2969
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 106s 28ms/step - accuracy: 0.0742 - loss: 0.9726 - val_accuracy: 0.0682 - val_loss: 1.2909


In [108]:
def traducir(oracion_ingles):
    traduccion = ""
    for id_palabra in range(long_max):
        X = tf.constant([oracion_ingles])  # codificar entrada
        X_dec = tf.constant(["startseq " + traduccion])  # decodificar entrada
        y_proba = modelo.predict((X, X_dec))[0, id_palabra]  # probabilidad del último token
        id_palabra_predicha = np.argmax(y_proba)
        palabra_predicha = capa_vect_esp.get_vocabulary()[id_palabra_predicha]
        if palabra_predicha == "endseq":
            break
        traduccion += " " + palabra_predicha
    return traduccion.strip()

In [75]:
traducir("I like soccer")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


'me gusta el fútbol'

## Ejemplo Modelo Seq2Seq con atención

In [81]:
#Usamos una capa Bidireccional para el codificador
codificador = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

In [82]:
# Asegúrate de que salidas_cod tenga la forma adecuada
salidas_cod_exp = tf.keras.layers.Lambda(
    lambda x: tf.expand_dims(x, axis=1),  # Expande para tener una dimensión de secuencia
    output_shape=(1, salidas_cod.shape[-1])  # Define la forma de salida
)(salidas_cod)

salidas_dec_exp = tf.keras.layers.Lambda(
    lambda x: tf.expand_dims(x, axis=1),
    output_shape=(1, salidas_dec.shape[-1])
)(salidas_dec)

# Capa de atención
capa_atencion = tf.keras.layers.Attention()

# Calcula la salida de atención
salida_atencion = capa_atencion([salidas_dec_exp, salidas_cod_exp])

# Agrega la capa de salida
capa_salida = tf.keras.layers.Dense(tam_voc, activation="softmax")
Y_proba = capa_salida(salida_atencion)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'lambda_16' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [88]:
traducir("I like play soccer with my friends")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'me gusta jugar al tenis con mis amigos'

## Ejemplo Modelo con Transformer

In [110]:
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.models import Model

In [111]:
long_max = 70  # maxima longitud de palabra
tam_emb = 128
tf.random.set_seed(42)
pos_capa_emb = tf.keras.layers.Embedding(long_max, tam_emb)
max_long_lote_cod = tf.keras.backend.int_shape(embeddings_cod)[1]

ent_codif = embeddings_cod + pos_capa_emb(tf.range(max_long_lote_cod))
max_long_lote_dec = tf.keras.backend.int_shape(embeddings_dec)[1]

ent_decodif = embeddings_cod + pos_capa_emb(tf.range(max_long_lote_dec))

In [112]:
# Funciones auxiliares para codificación posicional
def obtener_angulos(pos, i, d_model):
    angulos = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angulos

def codificacion_posicional(posicion, d_model):
    radianes = obtener_angulos(np.arange(posicion)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    radianes[:, 0::2] = np.sin(radianes[:, 0::2])
    radianes[:, 1::2] = np.cos(radianes[:, 1::2])
    pos_codificacion = radianes[np.newaxis, ...]
    return tf.cast(pos_codificacion, dtype=tf.float32)

In [113]:
#d_model = 128
entradas = Input(shape=(None,))
etiquetas = Input(shape=(None,))

# Embedding y codificación posicional para el codificador
cod_pos_cod = codificacion_posicional(long_max, tam_emb)
embeddings_cod += cod_pos_cod[:tf.keras.backend.int_shape(embeddings_cod)[1], :]

# Embedding y codificación posicional para el decodificador
dec_pos_encoding = codificacion_posicional(long_max, tam_emb)
embeddings_dec += dec_pos_encoding[:tf.keras.backend.int_shape(embeddings_dec)[1], :]

In [114]:
#Codificador
N = 2
num_cabezas = 8
tasa_dropout = 0.1
n_unidades = 128

# se encapsula la operación en una capa landa que es compatible con un modelo simbólica
masc_cod = tf.keras.layers.Lambda(
    lambda x: tf.math.not_equal(x, 0)[:, tf.newaxis]
)(ids_ent_cod)

Z = embeddings_cod
for _ in range(N):
    salto = Z
    capa_atencion = tf.keras.layers.MultiHeadAttention(
        num_heads=num_cabezas, key_dim=tam_emb, dropout=tasa_dropout)
    Z = capa_atencion(Z, value=Z, attention_mask=masc_cod)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, salto]))
    salto = Z
    Z = tf.keras.layers.Dense(n_unidades, activation="relu")(Z)
    Z = tf.keras.layers.Dense(tam_emb)(Z)
    Z = tf.keras.layers.Dropout(tasa_dropout)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, salto]))

In [115]:
#decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
masc_dec = tf.keras.layers.Lambda(
    lambda x: tf.math.not_equal(x, 0)[:, tf.newaxis]
)(ids_ent_dec)

causal_mask = tf.linalg.band_part(  # Matriz triangular inferior
    tf.ones((max_long_lote_dec, max_long_lote_cod), tf.bool), -1, 0)

In [116]:
# Decodificador
salida_cod = Z  # Se almacena la salida final de codificador
Z = embeddings_dec  # El decodificador inicar con su entrada propia
for _ in range(N):
    salto = Z
    capa_atencion = tf.keras.layers.MultiHeadAttention(
        num_heads=num_cabezas, key_dim=tam_emb, dropout=tasa_dropout)
    Z = capa_atencion(Z, value=Z, attention_mask=causal_mask & masc_dec)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, salto]))
    salto = Z
    capa_atencion = tf.keras.layers.MultiHeadAttention(
        num_heads=num_cabezas, key_dim=tam_emb, dropout=tasa_dropout)
    Z = capa_atencion(Z, value=salida_cod, attention_mask=masc_cod)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, salto]))
    salto = Z
    Z = tf.keras.layers.Dense(n_unidades, activation="relu")(Z)
    Z = tf.keras.layers.Dense(tam_emb)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, salto]))

In [117]:
Y_proba = tf.keras.layers.Dense(tam_voc, activation="softmax")(Z)
modelo = tf.keras.Model(inputs=[ent_cod, ent_dec],
                       outputs=[Y_proba])
modelo.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [118]:
modelo.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 277s 69ms/step - accuracy: 0.9402 - loss: 0.3696 - val_accuracy: 0.9641 - val_loss: 0.1562
Epoch 2/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 259s 69ms/step - accuracy: 0.9646 - loss: 0.1531 - val_accuracy: 0.9681 - val_loss: 0.1339
Epoch 3/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 263s 69ms/step - accuracy: 0.9678 - loss: 0.1344 - val_accuracy: 0.9695 - val_loss: 0.1269
Epoch 4/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 262s 70ms/step - accuracy: 0.9694 - loss: 0.1250 - val_accuracy: 0.9707 - val_loss: 0.1212
Epoch 5/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 261s 69ms/step - accuracy: 0.9706 - loss: 0.1186 - val_accuracy: 0.9711 - val_loss: 0.1176
Epoch 6/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 269s 71ms/step - accuracy: 0.9715 - loss: 0.1136 - val_accuracy: 0.9720 - val_loss: 0.1138
Epoch 7/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 313s 69ms/step - accuracy: 0.9721 - loss: 0.1101 - val_accuracy: 0.9725 - val_loss: 0.1121
Epoch 8/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 263s 69ms/step - accuracy: 

In [120]:
traducir("I like soccer and studying at home")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


'me gusta el fútbol y estudiar en casa'